# Description

##### The OOF of the model named "model_best_fold_0_imu_v4";
##### The inference process is: (1) prediction at wifi observations, (ii) interpolation these prediction to the true waypoint observation, (iii) Saito's post-processing + magnetic-fields based step adjustment, (iv) push-to-corridor post-processing;

##### This version: 
* $\alpha = 7.2, \beta = 0.3$;
* Use a modified step-heading estimating function.

##### This model score profile is:
* OOF CV before interpolation and before post-processing (MPE at wifi observations): 6.358;
* OOF CV after interpolation and before post-processing (MPE at waypoint observations): 6.773;
* Public LB after interpolation and before post-processing: 6.461;
* OOF CV after interpolation and after post-processing: ;
* Public LB after interpolation and after post-processing: 3.937;
* Moving average of headings, time window: 50 steps (1 second).

# Necessary packages

In [ ]:
import os
from glob import glob
import numpy as np
import math
import pandas as pd
from tqdm.notebook import tqdm
import copy
import pickle as pkl
import json
import matplotlib.pyplot as plt
import multiprocessing

import scipy
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
from sklearn.model_selection import GroupKFold

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW, lr_scheduler

import gc
import warnings
warnings.filterwarnings('ignore')

# Import data

In [ ]:
ts_data = glob('../input/iln-interpolate-waypoints-using-imu-data/*.csv')
ts = {}
for site in tqdm(ts_data):
    df = pd.read_csv(site, usecols = ['wifi_ts'])
    site = site.split('/')[-1].split('_')[0]
    ts[site] = df.values

In [ ]:
data_dir = '../input/indoor-with-delta'
files = glob(f'{data_dir}/*_train.csv')

train_data = pd.read_pickle(os.path.join(data_dir, 'train_all.pkl'))
train_data['timestamp'] = np.zeros(train_data.shape[0], dtype = int)

* Add timestamp

In [ ]:
ts = []
for file in tqdm(files):
    df = pd.read_csv(file, usecols = ['wifi_ts', 'path']).sort_values('wifi_ts')
    ts.append(df)
    
ts = pd.concat(ts)
ts = ts.groupby('path').apply(lambda x: x['wifi_ts'].values).to_dict()

In [ ]:
timestamp = []

for path in tqdm(train_data['path'].unique()):
    timestamp.append(ts[path])
train_data['timestamp'] = np.concatenate(timestamp)

* Format the data

In [ ]:
train_data.rename(columns = {'site': 'building'}, inplace = True)
train_data['group'] = train_data['path'].map(dict(zip(train_data['path'].unique(), range(train_data['path'].nunique()))))
train_data['site_path_timestamp'] = ['_'.join([i,j,str(k)]) for i, j, k in zip(train_data.building, train_data.path, train_data.timestamp)]
train_data.head()

In [ ]:
print('The number of NaN values is:')
print(train_data.isna().any().any())
print('Data information:')
train_data.info()

# Wifi encoder and building map

In [ ]:
with open('../input/iln-wifi-and-building-mapping/label_encoder_bssid.pkl', 'rb') as f:
    lbl_bssid = pkl.load(f)
    
with open('../input/iln-wifi-and-building-mapping/building_map.json', 'r') as f:
    building_map = json.load(f)

bssid_map = dict(zip(lbl_bssid.classes_, lbl_bssid.transform(lbl_bssid.classes_)))

In [ ]:
# Encode BSSID
bssid_features = [i for i in train_data.columns if i.startswith('bssid_')]
rssi_features = [i for i in train_data.columns if i.startswith('rssi_')]
timegap_features = [i for i in train_data.columns if i.startswith('gap_')]
train_data[bssid_features] = train_data[bssid_features].applymap(lambda x: bssid_map[x])

In [ ]:
train_data[bssid_features][train_data[rssi_features] == -999] = 0

In [ ]:
# Encode building
train_data['building'] = train_data.building.map(building_map)
train_data.head()

* IMU data

In [ ]:
imu_data = pd.read_pickle('../input/ilnaggregated-imu/train_imu_all.pkl')
imu_data['path'] = [i.split('_')[-1] for i in imu_data['site_floor_path']]
imu_data.head()

* Add user ID

In [ ]:
users = pd.read_csv('../input/retrieving-user-id-from-leaked-wifi-feature/df.csv', usecols = ['path_id', 'user_id'])
user_map = dict(zip(users['path_id'], users['user_id']))

In [ ]:
train_data['user_id'] = train_data['path'].map(user_map)
train_data.head()

# Feature extraction

In [ ]:
def feature_extraction_wifi(x):
    # Index
    idx = x.index.values.astype(int)
    
    # Path ID
    path = x['path'].unique()[0]
    
    # Building
    building = x['building'].unique().astype(int)
    
    # User ID
    user = x['user_id'].unique().astype(int)
    
    # BSSID
    bssid_feature = x[bssid_features].values.astype(int)
    
    # RSSI
    rssi_feature = x[rssi_features].values.astype(float)
    for i in range(len(rssi_features)):
        rssi_feature[:,i] = gaussian_filter1d(rssi_feature[:,i], sigma = 2)
    
    # Timegap
    timegap_feature = x[timegap_features].values.astype(float)
    
    # Delta waypoints
    del_waypoints = x[['delta_x_hat', 'delta_y_hat']].values.astype(float)
    
    # Waypoints
    waypoints = x[['x', 'y']].values.astype(float)
    
    # Floor
    floor = x['floor'].unique().astype(int) + 2
    
    return path, idx, building, user, bssid_feature, rssi_feature, timegap_feature, del_waypoints, waypoints, floor

In [ ]:
# Raw IMU
acce_ = ['acce_x', 'acce_y', 'acce_z']
gyro_ = ['gyro_x', 'gyro_y', 'gyro_z']
magn_ = ['magn_x', 'magn_y', 'magn_z']
ahrs_ = ['ahrs_x', 'ahrs_y', 'ahrs_z']
imu_ = acce_ + gyro_ + magn_ + ahrs_

def feature_extraction_imu(df):
    imu = df[imu_].values.astype(int)
    return imu

imu_group = imu_data.groupby('path').apply(feature_extraction_imu)

* Drop the Wifi data points that do not have the corresponding IMU data

In [ ]:
drop_path = train_data.path.unique()[np.in1d(train_data.path.unique(), np.array(imu_group.index))]
train_data = train_data[train_data['path'].isin(drop_path.tolist())].reset_index(drop = True)

# Dataset

In [ ]:
class ILN_Dataset(Dataset):
    def __init__(self, group, imu_group, max_len_wifi = 512, max_len_imu = 25_000):
        self.group = group
        self.imu_group = imu_group
        self.max_len_wifi = max_len_wifi
        self.max_len_imu = max_len_imu
        
    def __len__(self):
        return len(self.group)
    
    def __getitem__(self, idx):
        # Extract sequences
        path, idx, building, user, bssid_feature, rssi_feature, timegap_feature, del_waypoints, waypoints, floor = self.group.iloc[idx]
        
        # Load IMU data
        imu = self.imu_group.loc[path]
        
        seq_len_imu = imu.shape[0]
        mask_imu = np.ones(seq_len_imu)
        mask_imu_ = np.zeros(self.max_len_imu)
        
        # Sequence length
        seq_len_wifi = bssid_feature.shape[0]
        mask = np.ones(seq_len_wifi)
        
        mask_ = np.zeros(self.max_len_wifi)
        idx_ = np.zeros(self.max_len_wifi)
        bssid_feature_ = np.zeros((self.max_len_wifi, bssid_feature.shape[1]))
        rssi_feature_ = np.zeros((self.max_len_wifi, rssi_feature.shape[1]))
        timegap_feature_ = np.zeros((self.max_len_wifi, timegap_feature.shape[1]))
        del_waypoints_ = np.zeros((self.max_len_wifi, del_waypoints.shape[1]))
        waypoints_ = np.zeros((self.max_len_wifi, waypoints.shape[1]))
        
        imu_ = np.zeros((self.max_len_imu, imu.shape[1]))
        
        if seq_len_wifi <= self.max_len_wifi:   # Pad
            mask_[-seq_len_wifi:] = mask
            idx_[-seq_len_wifi:] = idx
            bssid_feature_[-seq_len_wifi:,:] = bssid_feature
            rssi_feature_[-seq_len_wifi:,:] = rssi_feature
            timegap_feature_[-seq_len_wifi:,:] = timegap_feature
            del_waypoints_[-seq_len_wifi:,:] = del_waypoints
            waypoints_[-seq_len_wifi:,:] = waypoints
        else:    # Cut
            mask_ = mask[-self.max_len_wifi:]
            idx_ = idx[-self.max_len_wifi:]
            bssid_feature_ = bssid_feature[-self.max_len_wifi:,:]
            rssi_feature_ = rssi_feature[-self.max_len_wifi:,:]
            timegap_feature_ = timegap_feature[-self.max_len_wifi:,:]
            del_waypoints_ = del_waypoints[-self.max_len_wifi:,:]
            waypoints_ = waypoints[-self.max_len_wifi:,:]
        
        if seq_len_imu > 0:
            if seq_len_imu <= self.max_len_imu:   # Pad
                mask_imu_[-seq_len_imu:] = mask_imu
                imu_[-seq_len_imu:,:] = imu
            else:    # Cut
                mask_imu_ = mask_imu[-self.max_len_imu:]
                imu_ = imu[-self.max_len_imu:,:]
            
        return {
            'mask': torch.tensor(mask_, dtype = torch.bool),
            'mask_imu': torch.tensor(mask_imu_, dtype = torch.bool),
            'idx': torch.tensor(idx_, dtype = torch.int),
            'imu': torch.tensor(imu_, dtype = torch.float),
            'building': torch.tensor(building, dtype = torch.long),
            'user': torch.tensor(user, dtype = torch.long),
            'bssid': torch.tensor(bssid_feature_, dtype = torch.long),
            'rssi': torch.tensor(rssi_feature_, dtype = torch.float),
            'timegap': torch.tensor(timegap_feature_, dtype = torch.float),
            'del_waypoints': torch.tensor(del_waypoints_, dtype = torch.float),
            'waypoints': torch.tensor(waypoints_, dtype = torch.float),
            'floor': torch.tensor(floor, dtype = torch.long)
        }

# Model

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % nhead == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // nhead
        self.nhead = nhead
        self.linears = clones(nn.Linear(d_model, d_model, bias=False), 4) # Q, K, V, last
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.nhead * self.d_k)
        return self.linears[-1](x)


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
class EncoderLayer(nn.Module):
    """
    Single Encoder block of SAINT
    """
    def __init__(self, d_model, nhead, dim_feedforward = 1024, dropout = 0.1):
        super().__init__()
        self._self_attn = MultiHeadedAttention(d_model, nhead, dropout)
        self._ffn = PositionwiseFeedForward(d_model, dim_feedforward, dropout)
        self._layernorms = clones(nn.LayerNorm(d_model, eps=1e-6), 2)
        self._dropout = nn.Dropout(dropout)

    def forward(self, src, mask = None):
        """
        query: question embeddings
        key: interaction embeddings
        """
        # self-attention block
        src2 = self._self_attn(query=src, key=src, value=src, mask=mask)
        src = src + self._dropout(src2)
        src = self._layernorms[0](src)
        src2 = self._ffn(src)
        src = src + self._dropout(src2)
        src = self._layernorms[1](src)
        return src

In [ ]:
class Lin_MultiHeadedAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1, max_len = 512, target_len = 400):
        "Take in model size and number of heads."
        super(Lin_MultiHeadedAttention, self).__init__()
        assert d_model % nhead == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // nhead
        self.nhead = nhead
        self.linears = clones(nn.Linear(d_model, d_model, bias=False), 4) # Q, K, V, last
        self.linear_key_value = clones(nn.Linear(max_len, 256, bias= False), 3)
        self.linear_query = nn.Linear(max_len, target_len)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(-1)
            # Mask query, key and value
            query = query * mask
            key = key * mask
            mask = value * mask
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Linear projections
        key, value = \
            [l(x).transpose(-1, -2)
             for l, x in zip(self.linear_key_value, (key.transpose(-1,-2), value.transpose(-1,-2)))]
        
        query = self.linear_query(query.transpose(-1, -2)).transpose(-1, -2)

        # 3) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=None,
                                 dropout=self.dropout)

        # 4) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.nhead * self.d_k)
        return self.linears[-1](x)
    
class Lin_EncoderLayer(nn.Module):
    """
    Single Encoder block of SAINT
    """
    def __init__(self, d_model, nhead, dim_feedforward = 1024, dropout = 0.1, max_len = 512, target_len = 400):
        super().__init__()
        self._self_attn = Lin_MultiHeadedAttention(d_model, nhead, dropout = dropout, max_len = max_len, target_len = target_len)
        self._ffn = PositionwiseFeedForward(d_model, dim_feedforward, dropout)
        self._layernorms = clones(nn.LayerNorm(d_model, eps=1e-6), 2)
        self._dropout = nn.Dropout(dropout)

    def forward(self, src, mask = None):
        """
        query: question embeddings
        key: interaction embeddings
        """
        # self-attention block
        src2 = self._self_attn(query=src, key=src, value=src, mask=mask)
        src = self._dropout(src2)
        src = self._layernorms[0](src)
        src2 = self._ffn(src)
        src = src + self._dropout(src2)
        src = self._layernorms[1](src)
        return src

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.max_len = max_len
        self.dropout = nn.Dropout(p=dropout)
        self.div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

    def forward(self, x, mask = None):
        if mask is not None:
            position = torch.cumsum(mask.unsqueeze(-1), dim = 1)
        else:
            position = torch.repeat_interleave(torch.arange(0, self.max_len).unsqueeze(-1).unsqueeze(0), x.shape[0], dim = 0)
        
        pe = torch.zeros(x.shape).to(x.device)
        div_term = self.div_term.to(x.device)
        pe[:,:,0::2] = torch.sin(position * div_term)
        pe[:,:,1::2] = torch.cos(position * div_term)
        
        x = x + pe
        
        return self.dropout(x)

In [ ]:
class ILN_Transformer(nn.Module):
    def __init__(self, num_feature_bssid = 100, num_feature_rssi = 100, num_building = 24, num_bssid = 216210, num_user = 27551, 
                 num_floor = 11, num_feature_imu = 12, d_model = 512, nhead = 4, max_len = 512, max_len_imu = 25_000, droprate = 0.1):
        super().__init__()
        self.num_feature_bssid = num_feature_bssid
        self.num_feature_rssi = num_feature_rssi
        self.num_building = num_building
        self.num_bssid = num_bssid
        self.num_user = num_user
        self.num_floor = num_floor
        self.d_model = d_model
        self.nhead = nhead
        self.max_len = max_len
        self.max_len_imu = max_len_imu
        self.droprate = droprate
        
        ############################################ Wifi ############################################
        # Embedding layers
        self.building_embedding = nn.Embedding(num_embeddings = self.num_building, embedding_dim = self.d_model)
        self.user_embedding = nn.Embedding(num_embeddings = self.num_user, embedding_dim = self.d_model)
        self.bssid_embedding = nn.Embedding(num_embeddings = self.num_bssid, embedding_dim = self.d_model // 2, padding_idx = 0)
        
        # Linear layers for BSSID
        self.linear_bssid = nn.Linear(self.num_feature_bssid, 1)
        
        # Linear layers for RSSI and timegap
        self.linear_rssi_timegap = nn.Linear(2, 1)
        self.linear_rssi = nn.Linear(self.num_feature_rssi, self.d_model // 2)
        self.layer_norm_rssi = nn.LayerNorm(self.d_model // 2)
        self.dropout_rssi = nn.Dropout(droprate)
        
        ############################################ IMU ############################################
        # Linear projection
        self.linear_imu_rotation = nn.Sequential(
            nn.Linear(num_feature_imu, d_model // 2),
            nn.LayerNorm(d_model // 2),
            nn.ReLU(),
            nn.Dropout(droprate)
        )
        
        # Positional encoder
        self.position_imu = PositionalEncoding(d_model = self.d_model // 2, dropout = self.droprate, max_len = self.max_len_imu)   # Because the IMU sequences are super long, we need a smaller model size to fit to the memory capacity
        
        # Attention
        self.attn_encoder_imu = Lin_EncoderLayer(d_model = self.d_model // 2, nhead = self.nhead, dim_feedforward = 256, dropout = self.droprate, max_len = self.max_len_imu, target_len = self.max_len)
        
        # self.shrink_time = nn.Linear(self.max_len_imu, self.max_len)
        
        # Output layer
        self.imu_output = nn.LSTM(input_size = d_model // 2, hidden_size = d_model // 2, bidirectional = True, batch_first = True)
        
        ############################################ Concatenate and prediction ############################################
        # Middle linear layers
        self.middle_linear = nn.Sequential(
            nn.Linear(2 * d_model, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(droprate)
        )
        
        # LSTM
        self.lstms = clones(nn.LSTM(input_size = self.d_model, hidden_size = self.d_model // 2, bidirectional = True, batch_first = True), 3)
        
        # Positional encoder
        self.position = PositionalEncoding(d_model = self.d_model, dropout = self.droprate, max_len = self.max_len)
        
        # Attention
        self.attn_encoder = EncoderLayer(d_model = self.d_model, nhead = self.nhead, dim_feedforward = 512, dropout = self.droprate)
        
        self.lstm = nn.LSTM(input_size = self.d_model + 2, hidden_size = self.d_model // 2, bidirectional = True, batch_first = True)
        self.linear_after_encoder = nn.Linear(self.d_model, self.d_model)
        self.layernorm_after_encoder = nn.LayerNorm(self.d_model)
        
        # Output
        self.linear_waypoints = nn.Linear(self.d_model, 2)
        
        # Activation
        self.softmax = nn.Softmax()
        self.prelu = nn.PReLU()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(self.droprate)
        
    def _get_pad_mask(self, seq, pad_idx):
        return (seq == pad_idx).unsqueeze(-2).to(seq.device)
    
    def _get_subsequent_mask(self, seq):
        sz_b, len_s = seq.size()
        subsequent_mask = torch.triu(torch.ones((1, len_s, len_s), device = seq.device), diagonal = 1).bool()
        return subsequent_mask
        
    def forward(self, mask, mask_imu, building, user, bssid, rssi, timegap, imu, del_waypoints):
        ############################################ Wifi ############################################
        # Embed
        building = self.building_embedding(building)
        user = self.user_embedding(user)
        bssid = self.bssid_embedding(bssid)
        
        # BSSID
        bssid = self.linear_bssid(bssid.transpose(-2,-1)).squeeze(-1)
        
        # RSSI and timegap
        rssi = torch.cat((rssi.unsqueeze(-1), timegap.unsqueeze(-1)), dim = -1)
        rssi = self.linear_rssi_timegap(rssi).squeeze(-1)
        rssi = self.linear_rssi(rssi)
        rssi = self.layer_norm_rssi(rssi)
        rssi = self.dropout_rssi(rssi)
        
        ############################################ IMU ############################################
        # Projection
        imu = self.linear_imu_rotation(imu)
        
        # Positional embedding
        imu = self.position_imu(imu, mask = mask_imu)
        
        # Attention
        imu = self.attn_encoder_imu(imu, mask = mask_imu)
        
        # Shrink time
        # imu = self.shrink_time(imu.transpose(-1,-2)).transpose(-1,-2)
        
        # Output IMU
        imu, _ = self.imu_output(imu)
        
        ############################################ Concatenate and prediction ############################################
        x_wifi = torch.cat((bssid, rssi), dim = -1)
        
        x = self.middle_linear(torch.cat((bssid, rssi, imu), dim = -1)) + building + user
        
        for i, layer in enumerate(self.lstms):
            x, _ = layer(x)
        
        # Positional encoding
        x = self.position(x, mask = mask)
        
        # Mask
        mask = ~(self._get_pad_mask(mask, False) | self._get_subsequent_mask(mask))
        
        # Feed it to the Encoder
        x = self.attn_encoder(x, mask)
        
        # Feed it to the lstm
        x, _ = self.lstm(torch.cat((x + x_wifi + imu, del_waypoints), dim = -1))
        
        # Feed over one more linear layer
        x = self.linear_after_encoder(self.dropout(self.prelu(x)))
        x = self.layernorm_after_encoder(x)
        x = self.dropout(self.prelu(x))
        
        # Concatenate floor output and x + building
        x = self.linear_waypoints(x)
        
        return x

# Configuration and util functions

* Metrics

In [ ]:
def metrics(output_way, output_floor, way, floor):
    first_term = np.mean(np.sqrt(np.sum((output_way - way)**2, axis = 1)))
    second_term = 15 * np.mean(np.abs(output_floor - floor))
    return first_term, second_term

def compute_cv_score(oof_):
    output_way = oof_[['pred_x', 'pred_y']].values
    output_floor = oof_['pred_floor'].values
    
    way = oof_[['true_x', 'true_y']].values
    floor = oof_['true_floor'].values
    
    loss_waypoints, loss_floor = metrics(output_way, output_floor, way, floor)
    return loss_waypoints, loss_floor

* Inference functions

In [ ]:
def infer_fn(model, infer_dataloader, device = 'cpu'):
    model.eval()
    
    index = []
    
    way_pred = []
    
    way_true = []
    floor_true = []
    
    loss = 0
    
    # tbar = tqdm(infer_dataloader)
    for item in infer_dataloader:
        padding_mask = item['mask'].to(device)
        padding_mask_imu = item['mask_imu'].to(device)
        imu = item['imu'].to(device)
        building = item['building'].to(device)
        user = item['user'].to(device)
        idx = item['idx'].to(device)
        bssid = item['bssid'].to(device)
        rssi = item['rssi'].to(device)
        timegap = item['timegap'].to(device)
        del_waypoints = item['del_waypoints'].to(device)
        waypoints = item['waypoints'].to(device)
        floor = item['floor'].to(device)
        
        # Feed input to the model
        with torch.no_grad():
            output_waypoints = model(padding_mask, padding_mask_imu, building, user, bssid, rssi, timegap, imu, del_waypoints)
        
        # True floor
        floor = torch.repeat_interleave(floor.unsqueeze(1), waypoints.shape[1], dim = 1)
        
        # Extract waypoints
        output_waypoints = output_waypoints[padding_mask,:]
        waypoints = waypoints[padding_mask,:]
        
        floor = floor[padding_mask,:] - 2
        
        idx = idx[padding_mask]
        
        # Store results
        index.append(idx.cpu().detach().numpy())
        
        way_pred.append(output_waypoints.cpu().detach().numpy())
        way_true.append(waypoints.cpu().detach().numpy())
        
        floor_true.append(floor.cpu().detach().numpy())
        
    # Stack
    index = np.concatenate(index)
    
    way_pred = np.vstack(way_pred)
    way_true = np.vstack(way_true)
    
    floor_true = np.vstack(floor_true)
    
    oof_ = pd.DataFrame(np.hstack((way_true, floor_true, way_pred, floor_true)), index = index, 
                        columns = ['true_x', 'true_y', 'true_floor', 'pred_x', 'pred_y', 'pred_floor'])
    
    return oof_

* Configuration

In [ ]:
class config():
    mode = 'infer'
    # For training
    split = GroupKFold(n_splits = 5)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # For dataloader
    batch_size = 64
    num_workers = 4
    # For model
    N = 2    # Number of encoder layers
    d_model = 256
    nhead = 2
    max_len = 400
    max_len_imu = 1024
    droprate = 0.3
    
cfg = config()

# Main

In [ ]:
model_path = '../input/iln-models'
oof_ = []
    
for i, (trn_idx, val_idx) in enumerate(tqdm(cfg.split.split(train_data.iloc[:,:200], 
                                                            train_data.building, 
                                                            train_data.group))):
    
    val = train_data.iloc[val_idx].copy()
    
    # Dataloader
    infer_group = val.groupby('group').apply(feature_extraction_wifi)
    infer_dataset = ILN_Dataset(infer_group, imu_group, max_len_wifi = cfg.max_len, max_len_imu = cfg.max_len_imu)
    infer_dataloader = DataLoader(infer_dataset, batch_size = cfg.batch_size, num_workers = cfg.num_workers, shuffle = False)
    
    ckp = torch.load(os.path.join(model_path, f'model_best_fold_{i}_imu_v4.pt'), map_location = cfg.device)
    model = ILN_Transformer(d_model = cfg.d_model, nhead = cfg.nhead, max_len_imu = cfg.max_len_imu,
                            max_len = cfg.max_len, droprate = cfg.droprate).to(cfg.device)
    model.load_state_dict(ckp['model_state_dict'])
    
    val_pred = infer_fn(model, infer_dataloader, device = cfg.device)
    
    val_pred['site_path_timestamp'] = val.loc[val_pred.index, 'site_path_timestamp'].values
    
    oof_.append(val_pred)

oof_ = pd.concat(oof_)
oof_ = oof_.sort_values('site_path_timestamp').reset_index(drop = True)

In [ ]:
oof_.to_csv('oof_wifi.csv', index = None)

* CV at wifi observations

In [ ]:
loss_waypoints, loss_floor = compute_cv_score(oof_)
print('OOF MPE waypoints:', loss_waypoints, ', OOF MPE floor:', loss_floor, ', OOF MPE:', loss_waypoints + loss_floor)

# Interpolate the prediction

In [ ]:
oof_[['site', 'path', 'timestamp']] = [i.split('_') for i in oof_.site_path_timestamp]
oof_['siteid_path'] = ['_'.join([i, j]) for i, j in zip(oof_['site'], oof_['path'])]
oof_.set_index('siteid_path', inplace = True)
oof_.head()

In [ ]:
train_waypoints = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
train_waypoints['site_path_timestamp'] = ['_'.join([i, j, str(k)]) for i, j, k in zip(train_waypoints.site, train_waypoints.path, train_waypoints.timestamp)]
ss = train_waypoints[['site_path_timestamp', 'floor', 'x', 'y', 'site', 'path', 'timestamp']]
ss.rename(columns = {'x': 'true_x', 'y': 'true_y'}, inplace = True)
ss['pred_floor'] = ss['floor'].values

samples = pd.DataFrame(ss.groupby(['site','path'])['timestamp'].apply(lambda x: list(x)))
buildings = oof_.site.unique()
samples.head()

In [ ]:
!git clone --depth 1 https://github.com/location-competition/indoor-location-competition-20 indoor_location_competition_20
!rm -rf indoor_location_competition_20/data

from indoor_location_competition_20.io_f import read_data_file
import indoor_location_competition_20.compute_f as compute_f

In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage.filters import uniform_filter1d

colacce = ['Unnamed: 0','acce_x','acce_y','acce_z']
colahrs = ['Unnamed: 0','ahrs_x','ahrs_y','ahrs_z']

error_paths = []

for building in buildings:
    print(building)
    paths = oof_[oof_.site == building]['path'].unique()
    # Acceleration info
    for path_id in paths:
        # Load IMU data of this path_id
        tfm = pd.read_csv(glob(f'../input/ilnimu-clean/{building}_*_{path_id}.csv')[0], usecols = ['Unnamed: 0'] + imu_)
        # Original predicted values:
        xy = oof_.loc[building + '_' + path_id]
        acce_datas = np.array(tfm[colacce],dtype = np.float)
        ahrs_datas = np.array(tfm[colahrs],dtype = np.float)
        posi_datas = np.array(xy[['timestamp', 'pred_x', 'pred_y']], dtype = np.float).reshape(-1,3)
        # Outlier removal:
        xyout = uniform_filter1d(posi_datas, size = 3, axis = 0, mode = 'reflect').reshape(-1,3)
        xydiff = np.abs(posi_datas - xyout)
        xystd = np.std(xydiff,axis = 0) * 3
        posi_datas = posi_datas[(xydiff [:,1] < xystd[1]) & (xydiff[:,2] < xystd[2])]
        # Step detection:
        step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
        stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
        # Orientation detection:
        headings = compute_f.compute_headings(ahrs_datas)
        step_headings = compute_f.compute_step_heading(step_timestamps, headings)
        rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
        # Running average:
        posi_datas = uniform_filter1d(posi_datas, size = 3, axis = 0, mode = 'reflect')[0::3,:]
        # The 1st prediction timepoint should be earlier than the 1st step timepoint.
        try:
            rel_positions = rel_positions[rel_positions[:,0] > posi_datas[0,0],:]
            # If two consecutive predictions are in-between two step datapoints,
            # the last one is removed, causing error (in the "split_ts_seq" function).
            posi_index = [np.searchsorted(rel_positions[:,0], x, side = 'right') for x in posi_datas[:,0]]
            u, i1, i2 = np.unique(posi_index, return_index = True, return_inverse = True)
            posi_datas = np.vstack([np.mean(posi_datas[i2 == i],axis = 0) for i in np.unique(i2)])
            # Position correction:
            step_positions = compute_f.correct_positions(rel_positions, posi_datas)
            # Interpolate for timestamps in the testing set:
            t = step_positions[:,0]
            x = step_positions[:,1]
            y = step_positions[:,2]
        
            fx = interp1d(t, x, kind = 'linear', fill_value = (x[0], x[-1]), bounds_error = False) #fill_value="extrapolate"
            fy = interp1d(t, y, kind = 'linear', fill_value = (y[0], y[-1]), bounds_error = False)
            # Output result:
            t0 = np.array(samples.loc[(building,path_id),'timestamp'], dtype = np.float64)

            ss.loc[(ss.site == building) & (ss.path == path_id), 'pred_x'] = fx(t0)
            ss.loc[(ss.site == building) & (ss.path == path_id), 'pred_y'] = fy(t0)
        except:
            error_paths.append(path_id)
            pass

ss = ss.dropna(axis = 0).rename(columns = {'floor': 'true_floor'}).reset_index(drop = True)
ss['timestamp'] = ss['timestamp'].astype(float)
ss.head()

In [ ]:
ss.to_csv('oof_waypoints_before_pp.csv', index = None)

* Compute the MPE CV

In [ ]:
loss_waypoints, loss_floor = compute_cv_score(ss)
print('OOF MPE waypoints:', loss_waypoints, ', OOF MPE floor:', loss_floor, ', OOF MPE:', loss_waypoints + loss_floor)

# Post-processing steps

* Visualization functions

In [ ]:
# Helper Functions
def split_col(df):
    df = pd.concat([
        df['site_path_timestamp'].str.split('_', expand=True) \
        .rename(columns={0:'site',
                         1:'path',
                         2:'timestamp'}),
        df
    ], axis=1).copy()
    return df

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}

def plot_preds(
    site,
    floorNo,
    sub=None,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=True,
    show_preds=True,
    show_smoothed=False,
    fix_labels=True,
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")

    fig, ax = plt.subplots(figsize=(12, 12))
    plt.imshow(floor_img)

    if show_train:
        true_locs = true_locs.query('site == @site and floorNo == @map_floor').copy()
        true_locs["x_"] = true_locs["x"] * floor_img.shape[0] / height_meter
        true_locs["y_"] = (
            true_locs["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        true_locs.query("site == @site and floorNo == @map_floor").groupby("path").plot(
            x="x_",
            y="y_",
            style="+",
            ax=ax,
            label="train waypoint location",
            color="grey",
            alpha=0.5,
        )

    if show_preds:
        if not show_smoothed:
            sub = sub.query('site == @site and floorNo == @floorNo').copy()
            sub["x_"] = sub["pred_x"] * floor_img.shape[0] / height_meter
            sub["y_"] = (
                sub["pred_y"] * -1 * floor_img.shape[1] / width_meter
            ) + floor_img.shape[0]
            for path, path_data in sub.query(
                "site == @site and floorNo == @floorNo"
            ).groupby("path"):
                path_data.plot(
                    x="x_",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{site} - floor - {floorNo}",
                    alpha=1,
                    label=path,
                )
                
        else:
            sub = sub.query('site == @site and floorNo == @floorNo').copy()
            sub["x_"] = sub["pred_x"] * floor_img.shape[0] / height_meter
            sub["y_"] = (
                sub["pred_y"] * -1 * floor_img.shape[1] / width_meter
            ) + floor_img.shape[0]
            for path, path_data in sub.query(
                "site == @site and floorNo == @floorNo"
            ).groupby("path"):
                path_data.plot(
                    x="x_",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{site} - floor - {floorNo}",
                    alpha=1,
                    label=path,
                )
                
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
        
    return fig, ax

def sub_process(sub, train_waypoints):
    train_waypoints['isTrainWaypoint'] = True
    sub = sub.rename(columns = {'pred_floor': 'floor', 'pred_x': 'x', 'pred_y': 'y'})
    #sub = split_col(sub).copy()
    sub = sub.merge(train_waypoints[['site','floorNo','floor']].drop_duplicates(), how='left')
    sub = sub.merge(
        train_waypoints[['x','y','site','floor','isTrainWaypoint']].drop_duplicates(),
        how = 'left',
        on = ['site','x','y','floor'])
    sub['isTrainWaypoint'] = sub['isTrainWaypoint'].fillna(False)
    sub = sub.rename(columns = {'floor': 'pred_floor', 'x': 'pred_x', 'y': 'pred_y'})
    return sub.copy()

* Visualization of raw prediction

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

* Saito's post-processing

In [ ]:
import math

order = 3
fs = 50.0  # sample rate, Hz
# fs = 100
# cutoff = 3.667  # desired cutoff frequency of the filter, Hz
cutoff = 3

step_distance = 0.8
w_height = 1.7
m_trans = -5

from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def peak_accel_threshold(data, timestamps, threshold):
    d_acc = []
    last_state = 'below'
    crest_troughs = 0
    crossings = []

    for i, datum in enumerate(data):
        
        current_state = last_state
        if datum < threshold:
            current_state = 'below'
        elif datum > threshold:
            current_state = 'above'

        if current_state is not last_state:
            if current_state is 'above':
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)
            else:
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)

            crest_troughs += 1
        last_state = current_state
    return np.array(crossings)

def steps_compute_rel_positions(sample_file):
    
    mix_acce = np.sqrt(sample_file.acce[:,1:2]**2 + sample_file.acce[:,2:3]**2 + sample_file.acce[:,3:4]**2)
    mix_acce = np.concatenate([sample_file.acce[:,0:1], mix_acce], 1)
    mix_df = pd.DataFrame(mix_acce)
    mix_df.columns = ["timestamp","acce"]
    
    filtered = butter_lowpass_filter(mix_df["acce"], cutoff, fs, order)

    threshold = filtered.mean() * 1.1
    crossings = peak_accel_threshold(filtered, mix_df["timestamp"], threshold)

    step_sum = len(crossings)/2
    distance = w_height * 0.4 * step_sum

    mag_df = pd.DataFrame(sample_file.magn)
    mag_df.columns = ["timestamp","x","y","z"]
    
    acce_df = pd.DataFrame(sample_file.acce)
    acce_df.columns = ["timestamp","ax","ay","az"]
    
    mag_df = pd.merge(mag_df,acce_df,on="timestamp")
    mag_df.dropna()
    
    time_di_list = []

    for i in mag_df.iterrows():

        gx,gy,gz = i[1][1],i[1][2],i[1][3]
        ax,ay,az = i[1][4],i[1][5],i[1][6]

        roll = math.atan2(ay,az)
        pitch = math.atan2(-1*ax , (ay * math.sin(roll) + az * math.cos(roll)))

        q = m_trans - math.degrees(math.atan2(
            (gz*math.sin(roll)-gy*math.cos(roll)),(gx*math.cos(pitch) + gy*math.sin(roll)*math.sin(pitch) + gz*math.sin(pitch)*math.cos(roll))
        )) -90
        if q <= 0:
            q += 360
        time_di_list.append((i[1][0],q))

    d_list = [x[1] for x in time_di_list]
    
    steps = []
    step_time = []
    di_dict = dict(time_di_list)

    for n,i in enumerate(crossings[:,:1]):
        if n % 2 == 1:
            continue
        direct_now = di_dict[i[0]]
        dx = math.sin(math.radians(direct_now))
        dy = math.cos(math.radians(direct_now))
#         print(int(n/2+1),"歩目/x:",dx,"/y:",dy,"/角度：",direct_now)
        steps.append((i[0],dx,dy))
        step_time.append(i[0])
    
        step_dtime = np.diff(step_time)/1000
        step_dtime = step_dtime.tolist()
        step_dtime.insert(0,5)
        
        rel_position = []

        wp_idx = 0
#         print("WP:",round(sample_file.waypoint[0,1],3),round(sample_file.waypoint[0,2],3),sample_file.waypoint[0,0])
#         print("------------------")
        for p,i in enumerate(steps):
            step_distance = 0
            if step_dtime[p] >= 1:
                step_distance = w_height*0.25
            elif step_dtime[p] >= 0.75:
                step_distance = w_height*0.3
            elif step_dtime[p] >= 0.5:
                step_distance = w_height*0.4
            elif step_dtime[p] >= 0.35:
                step_distance = w_height*0.45
            elif step_dtime[p] >= 0.2:
                step_distance = w_height*0.5
            else:
                step_distance = w_height*0.4

#             step_x += i[1]*step_distance
#             step_y += i[2]*step_distance
            
            rel_position.append([i[0], i[1]*step_distance, i[2]*step_distance])
#     print(rel_position)
    
    return np.array(rel_position)

In [ ]:
def compute_step_heading(step_timestamps, headings):
    sin_head = np.sin(headings[:,1])
    cos_head = np.cos(headings[:,1])
    head = np.vstack((np.searchsorted(step_timestamps, headings[:,0]), sin_head, cos_head)).T
    split_steps = np.split(head[:,1:], np.unique(head[:, 0], return_index = True)[1][1:])
    step_headings = np.array([np.sum(arr, axis = 0) for arr in split_steps[:-1]])
    step_headings = np.arctan2(step_headings[:,0], step_headings[:,1])
    step_headings = np.vstack((step_timestamps, step_headings)).T
    return step_headings

def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    headings[:,1] = uniform_filter1d(headings[:,1], size = 50)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)    # compute_f.
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

def correct_path(args):
    path, path_df = args
    
    site = path_df.site.unique()[0]
    floor = path_df.pred_floor.unique()[0]
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['pred_x', 'pred_y']].values
    
    # Find the file path
    floor_name = os.listdir('../input/indoor-location-navigation/train/' + site)

    for name in floor_name:
        if floor_map[name] == floor:
            file = '../input/indoor-location-navigation/train/' + site + '/' + name + '/' + path + '.txt'
            break
            
    try:
        example = read_data_file(file)
    
        rel_positions1 = compute_rel_positions(example.acce, example.ahrs)
        rel_positions2 = steps_compute_rel_positions(example)
        rel1 = rel_positions1.copy()
        rel2 = rel_positions2.copy()
        rel1[:,1:] = rel_positions1[:,1:] / 2
        rel2[:,1:] = rel_positions2[:,1:] / 2
        rel_positions = np.vstack([rel1,rel2])
        rel_positions = rel_positions[np.argsort(rel_positions[:, 0])]

        if T_ref[-1] > rel_positions[-1, 0]:
            rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
        else:
            rel_positions = [np.array([[0, 0, 0]]), rel_positions]
        rel_positions = np.concatenate(rel_positions)

        T_rel = rel_positions[:, 0]
        delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

        N = xy_hat.shape[0]
        delta_t = np.diff(T_ref)
        alpha = (7.2)**(-2) * np.ones(N)
        beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
        A = scipy.sparse.spdiags(alpha, [0], N, N)
        B = scipy.sparse.spdiags(beta, [0], N-1, N-1)
        D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

        Q = A + (D.T @ B @ D)
        c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
        xy_star = scipy.sparse.linalg.spsolve(Q, c)

        return pd.DataFrame({
            'site_path_timestamp' : path_df['site_path_timestamp'],
            'pred_floor' : path_df['pred_floor'],
            'pred_x' : xy_star[:, 0],
            'pred_y' : xy_star[:, 1],
            'true_floor' : path_df['true_floor'],
            'true_x' : path_df['true_x'],
            'true_y' : path_df['true_y'],
        })
    except:
        print('Mis-predict floor: ', file)
        return pd.DataFrame({
            'site_path_timestamp' : path_df['site_path_timestamp'],
            'pred_floor' : path_df['pred_floor'],
            'pred_x' : path_df['pred_x'],
            'pred_y' : path_df['pred_x'],
            'true_floor' : path_df['true_floor'],
            'true_x' : path_df['true_x'],
            'true_y' : path_df['true_y'],
        })

In [ ]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes = processes) as pool:
    dfs = pool.imap_unordered(correct_path, ss.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
ss = pd.concat(dfs).sort_values('site_path_timestamp').reset_index(drop = True)

In [ ]:
ss.to_csv('oof_waypoints_after_Saito.csv', index = None)

In [ ]:
loss_waypoints, loss_floor = compute_cv_score(ss)
print("After Saito's post-processing")
print('OOF MPE waypoints:', loss_waypoints, ', OOF MPE floor:', loss_floor, ', OOF MPE:', loss_waypoints + loss_floor)

tmp = ss['site_path_timestamp'].apply(lambda s : pd.Series(s.split('_')))
ss['site'] = tmp[0]
ss['path'] = tmp[1]
ss['timestamp'] = tmp[2].astype(float)

* Visualize post-processing after Saito's post processing

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

* Push to corridor post-processing

In [ ]:
from shapely.geometry import Polygon
from shapely.ops import nearest_points
from shapely.geometry import Point

def fix_prediction(args):
    # Unpack
    (site, floor), df = args
    
    # Find the file path
    floor_name = os.listdir('../input/indoor-location-navigation-scaled-geojson/scaled_geojson/' + site)
    for name in floor_name:
        if floor_map[name] == floor:
            file = '../input/indoor-location-navigation-scaled-geojson/scaled_geojson/' + site + '/' + name + '/shapely_geometry.pkl'
            break
            
    # Open the corridor
    with open(file, 'rb') as f:
        corridor = pkl.load(f)
        
    # Find the outside-corridor points and force them into the corridor
    out_corridor = []
    out_corridor_idx = []
    corridor_nearest_points = []
    for i in range(df.shape[0]):
        p = Point(df[['pred_x', 'pred_y']].iloc[i].values)
        if not p.within(corridor):
            out_corridor.append(p)
            out_corridor_idx.append(df[['pred_x', 'pred_y']].index[i])
            nearest_p, _ = nearest_points(corridor, p)
            x, y = nearest_p.xy[0][0], nearest_p.xy[1][0]
            corridor_nearest_points.append([x, y])
    
    if len(corridor_nearest_points) != 0:
        df.loc[out_corridor_idx, ['pred_x', 'pred_y']] = np.array(corridor_nearest_points)
    
    return df

In [ ]:
ss[['site', 'path', 'timestamp']] = np.array([i.split('_') for i in ss.site_path_timestamp])

processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes = processes) as pool:
    dfs = pool.imap_unordered(fix_prediction, ss.groupby(['site', 'pred_floor']))
    dfs = tqdm(dfs)
    dfs = list(dfs)
ss = pd.concat(dfs).sort_values('site_path_timestamp')

In [ ]:
ss.to_csv('oof_waypoints_after_pp.csv', index = None)

In [ ]:
loss_waypoints, loss_floor = compute_cv_score(ss)
print('After Push-to-corridor post-processing')
print('OOF MPE waypoints:', loss_waypoints, ', OOF MPE floor:', loss_floor, ', OOF MPE:', loss_waypoints + loss_floor)

* Visualize after Push-to-Corridor post-processing

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub, 
           train_waypoints, show_preds = True)
plt.show()